# Child and Dependent Care Credit

The Child and Dependent Care Tax Credit (CDCC) provides a share of care expenses depending on income.

## Examples

Consider a single parent with \$8,000 child care expenses per child (the CDCC's maximum in 2021, up to two children).
Since the CDCC is capped at the lower of head and spouse's earnings, this example is limited to single parents.
In 2020, the CDCC phased out only at one level; in 2021 only, it phased out twice, and in 2022 and beyond, it returns to 2020 law.

In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_cdcc(children, year):
    sim = IndividualSim(year=year)
    sim.add_person(name="head", is_tax_unit_head=True)
    members = ["head"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members += [child]
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
        tax_unit_childcare_expenses=8_000 * children,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MD")
    sim.vary("employment_income", max=500_000)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            cdcc=sim.calc("cdcc")[0].round(),
            cdcc_mtr=-sim.deriv(
                "cdcc",
                "employment_income",
                wrt_target="head",
            ),
            children=children,
            year=year,
        )
    )


# Make a table of CDCC amounts for different numbers of adults and children.
l = []
for year in [2020, 2021, 2022]:
    for children in range(1, 3):
        l.append(make_cdcc(children, year))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    cdcc="CDCC",
    cdcc_mtr="CDCC Marginal Tax Rate",
    children="Children",
    year="Year",
)

fig = px.line(
    df,
    "employment_income",
    "cdcc",
    color="children",
    animation_frame="year",
    labels=LABELS,
    title="Child and Dependent Care Credit",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.cdcc.max() * 1.05],
)
fig.show()

The credit phases in at 50% of income and then phases out in a stepwise fashion that produces many small cliffs (infinite marginal tax rates).

In [2]:
fig = px.line(
    df,
    "employment_income",
    "cdcc_mtr",
    color="children",
    animation_frame="year",
    labels=LABELS,
    title="CDCC marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[-1, 1]
)
fig.show()

## Budgetary impact

Applying 2022 rules to the 2020 Current Population Survey, OpenFisca US estimates that the CDCC pays out \$4.0 billion.

In [3]:
from policyengine_us import Microsimulation

sim = Microsimulation(year=2020)

sim.calc("cdcc", period=2022).sum() / 1e6

4014.26586928978

However, since it interacts with other programs, we need to compare total income after repealing the program to estimate the true budgetary impact.
Applying this method yields a lower \$3.8 billion.

In [4]:
from policyengine_us.model_api import *


class ignore_reported(Reform):
    def apply(self):
        self.neutralize_variable("spm_unit_net_income_reported")


class neutralize_cdcc(Reform):
    def apply(self):
        self.neutralize_variable("cdcc")


sim = Microsimulation(ignore_reported, year=2020)
sim_no_cdcc = Microsimulation((ignore_reported, neutralize_cdcc), year=2020)

(
    sim.calc("spm_unit_net_income", period=2022).sum()
    - sim_no_cdcc.calc("spm_unit_net_income", period=2022).sum()
) / 1e6

3769.459399326172